In [2]:
pip install fuzzywuzzy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from music21 import * 
from music21 import corpus, meter, tempo, expressions
import numpy as np
import pandas as pd
from collections import Counter
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Users/melodyqian/Library/Python/3.9/lib/python/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
folder_path = '/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/total57/'
paths=[]
filenames=[]
for filename in os.listdir(folder_path):
    if filename.endswith('.mxl'):  # Or the file extension you're working with
        full_path = os.path.join(folder_path, filename)
        paths.append(full_path)
        filenames.append(filename)

for i in range(len(filenames)):
    filenames[i] = filenames[i].replace('_', ' ')  
    filenames[i] = filenames[i].replace('.mxl', '')

pathsdict=dict(zip(filenames, paths))

In [5]:
json=pd.read_json("/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/scrape-musescore-2/sheet_music_results.json")
difficultymetric=json.dropna(subset=['title'])

In [6]:
def match_titles_to_files(df, filenames, dict, threshold=70):
    matched_files = []
    for title in df['title']:
        match = process.extractOne(title, filenames, scorer=fuzz.token_sort_ratio)
        if match and match[1] >= threshold: 
           matched_files.append(dict.get(match[0], np.nan))  
        else:
            matched_files.append(np.nan)  
    return matched_files  

In [7]:
filekeys = list(pathsdict.keys())
difficultymetric['matched_file'] = match_titles_to_files(difficultymetric, filekeys, pathsdict)

/var/folders/bf/j0nvm48j18b4mlfybgxf_1fm0000gn/T/ipykernel_9116/3898033698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  difficultymetric['matched_file'] = match_titles_to_files(difficultymetric, filekeys, pathsdict)


In [ ]:

difficultymetric[['title', 'matched_file']]
difficultymetric=difficultymetric.dropna(subset=['matched_file'])

str

In [45]:
note_durations = {
    '32nd_and_shorter': 0.125,
    '16th': 0.25,
    'eighth': 0.5,
    'quarter': 1,
    'dotted quarter': 1.5,
    'half': 2,
    'dotted half': 3,
    'whole': 4
}

In [213]:
def analyze(filepath):
    filepath=str(filepath)
    score=converter.parse(filepath)
    flatscore=score.flatten()
    def notecount(flatscore):
       num_notes=0
       for elem in flatscore:
           if isinstance(elem, note.Note):
              num_notes += 1
           elif isinstance(elem, chord.Chord):
              num_notes += len(elem.pitches)
       return(num_notes)
    notecounts=notecount(flatscore)
    chords= [element for element in flatscore if isinstance(element, chord.Chord)]
    numberchords= len(chords)   
    def chordpitchdistances(chords):
        pitchdistances= []
        for i in range(len(chords)-1):
            firstchord= chords[i]
            secondchord= chords[i+1]
            rootInterval=interval.Interval(firstchord.root(),secondchord.root())
            pitchdistances.append(rootInterval.semitones)
        return(pitchdistances)
    pitchdistances=chordpitchdistances(chords)
    chord_abs_dis= [abs(dis) for dis in pitchdistances]
    meanchorddis= np.mean(chord_abs_dis)
    def chordtimedistances(chords):
        timedistances=[]
        for i in range(len(chords)-1):
            firstchord= chords[i]
            secondchord= chords[i+1]
            time_difference = secondchord.offset - firstchord.offset
            timedistances.append(time_difference) 
        return timedistances
    chordtimedistance=chordtimedistances(chords)
    meantimedistance=np.mean(chordtimedistance)
    key=flatscore.analyze('key')
    timesignature=flatscore.getElementsByClass(meter.TimeSignature)
    timesignaturechanges=len(timesignature)
    def findtrills(score):
        trills=0
        totalbeats=0
        for element in score.notesAndRests:
            if element.expressions:  # Check if the note has expressions
                for expr in element.expressions:
                    if isinstance(expr, expressions.Trill):  # Check if it's a trill
                        trills+= 1 
                        totalbeats+=element.quarterLength
        return [trills, totalbeats]
    trill_length=findtrills(flatscore)
    tempos= flatscore.getElementsByClass(tempo.MetronomeMark)
    sorted_tempos = sorted(tempos, key=lambda tm: tm.measureNumber)
    weighted_tempo=[]
    for i in range(len(sorted_tempos)):
        current_bpm = sorted_tempos[i].getQuarterBPM()
        current_measure = sorted_tempos[i].measureNumber
        if i < len(sorted_tempos) - 1:
            next_measure = sorted_tempos[i + 1].measureNumber
            duration = next_measure - current_measure
        else:
            # If this is the last tempo, assume it lasts to the end of the piece
            duration = score.highestTime
        weighted_tempo.append((current_bpm, duration))
    total_weight = sum(duration for _, duration in weighted_tempo)
    weighted_mean = (
    sum(bpm * duration for bpm, duration in weighted_tempo) / total_weight
    if total_weight != 0 else 0
    )
    num_measures = len(score.recurse().getElementsByClass('Measure'))
    duration_counter = Counter(note.duration.quarterLength for note in score.recurse().notes)
    def count_notes(score, duration_counter):
        note_counts= {}
        for name, duration in note_durations.items():
            if name=='32nd_and_shorter':
                note_counts[name]= sum(count for d, count in duration_counter.items() if d<=duration)
            else:
                note_counts[name]=duration_counter[duration]
        return note_counts
    notedict= count_notes(score,duration_counter)
    return{
        "numberofnotes": notecounts,
        "numberofchords": numberchords,
        "numberofmeasures": num_measures,
        "meandistancebetweenchords": meanchorddis,
        "meantimebetweenchords": meantimedistance,
        "key": key,
        "timechanges": timesignaturechanges,
        "trillslength": trill_length,
        "avgtempo" : weighted_mean,
        **notedict
    }

In [ ]:
results_df = difficultymetric['matched_file'].apply(analyze).apply(pd.Series)

/Users/melodyqian/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/melodyqian/Library/Python/3.9/lib/python/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
results_df['Title'] = difficultymetric['title'].values

22                  Bach BWV 807 Allemande English Suite
23     J.S. Bach - English Suite no.3 - Allemande, BW...
29     J.S. Bach - French Suite No. 5 Gavotte in G Ma...
30              Goldberg Variations BWV 988, Variation 1
35     Bach JS - Partita No 2 in C minor BMV 826 Move...
40                       Fugue No. 16 BWV 885 in G Minor
41                     Prélude No. 20 BWV 865 in A Minor
42                     Prélude No. 20 BWV 889 in A Minor
43                      Fugue No. 17 BWV 862 in A♭ Major
44                      Fugue No. 17 BWV 886 in A♭ Major
46                       Fugue No. 23 BWV 868 in B Major
47                     Prélude No. 23 BWV 892 in B Major
48                       Fugue No. 24 BWV 869 in B Minor
49                     Prélude No. 24 BWV 893 in B Minor
50                       BWV 866 - Fugue in B flat Major
52                    Prélude No. 22 BWV 867 in B♭ Minor
53                      Fugue No. 22 BWV 891 in B♭ Minor
56                        Fugue

In [219]:
results_df.to_csv("Difficulty_Metric_Sample.csv", index=False) 